# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import random
import os
seed = 42
random.seed(seed)
np.random.seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)

In [3]:
sample = pd.read_csv('/kaggle/input/2022-ml-midterm-p1/submit.csv')
train = pd.read_csv('/kaggle/input/2022-ml-midterm-p1/train.csv')
test = pd.read_csv('/kaggle/input/2022-ml-midterm-p1/test.csv')

데이터를 불러오기 위해서 pd.read_csv() 함수를 사용하였다.

In [4]:
train

,index,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,0,63,1,1,145,233,1,2,150,0,2.3,3,0,6,0
1,1,67,1,4,160,286,0,2,108,1,1.5,2,3,3,2
2,2,67,1,4,120,229,0,2,129,1,2.6,2,2,7,1
3,4,41,0,2,130,204,0,2,172,0,1.4,1,0,3,0
4,5,56,1,2,120,236,0,0,178,0,0.8,1,0,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237,295,41,1,2,120,157,0,0,182,0,0.0,1,0,3,0
238,296,59,1,4,164,176,1,2,90,0,1.0,2,2,6,3
239,299,68,1,4,144,193,1,0,141,0,3.4,2,2,7,2
240,300,57,1,4,130,131,0,0,115,1,1.2,2,1,7,3


In [5]:
X_train = train.drop(['index','target'],axis=1)
train[train['target']>1]=1
y_train = train['target']
X_test = test.drop(['index'],axis=1)

데이터를 살펴보니 'index'가 공통적으로 훈련데이터와 학습데이터에 들어있어서 제거해주었고, 훈련데이터에서는 학습을 하기위해서 'target'라는 라벨을 따로 떼서 다른 변수에 담아주었다. 또한 문제에서 심장병 유무를 예측하는 것이므로 학습데이터중 정답라벨의 값들중 1보다 큰 값들을 전부 1로 변경해 주었다.

In [6]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression().fit(X_train, y_train)
y_test = model.predict(X_test)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


해당 문제는 심장질환 환자 예측하는 문제로 분류 문제에 해당하기에 먼저 분류모델을 만들어야한다고 생각했고 학습데이터의 수가 241개 정도로 적고 0과 1로 분류하는 문제여서 선형으로 구분할 수가 있다고 판단하여 LogisticRegression을 선언하고 학습을 시켰다.

In [7]:
sample['target'] = y_test
sample.to_csv('submit.csv',index=False)

모델을 통해서 에측한 값을 csv에 저장하기 위해서 .to_csv라는 함수를 사용했다.

In [8]:
from sklearn.ensemble import RandomForestClassifier
model1 = RandomForestClassifier().fit(X_train,y_train)
y_test1 = model1.predict(X_test)

해당 문제는 분류 문제로 분류모델을 만들어야겠다고 생각했고, 학습의 데이터의 수가 많고 설명보다는 예측을 필요로 하는 문제이기에 DecisionTreeClassifier보다는 RandomForestClassifier가 더 적합하다고 생각하여 RandomForestClassifier을 선언하고 학습을 시켰다

In [9]:
#sample['target'] = y_test1
#sample.to_csv('submit1.csv',index=False)

모델을 통해서 에측한 값을 csv에 저장하기 위해서 .to_csv라는 함수를 사용했다.

LogisticRegression으로 모델을 만들고 파라미터를 default 값으로 설정후 캐글에 제출해보니 0.85245 가 나왔고 RandomForestClassifier로 모델을 만들고 파라미터를 default 값으로 설정후 캐글에 제출해보니 0.83606 가 나왔다. 이를 통해서 분석을 해보자면 해당 데이터에서는 우선 학습데이터가 적어서 KNeighborsClassifier를 사용하지 않았고 해당 문제는 0과 1로 분류하는 문제여서 시그모이드 함수를 사용하는 LogisticRegression이 더 효과가 좋을 것이라고 생각을 했고 또한 해당 데이터는 선형으로 분류가 되는 문제라고 판단을 하였다. 따라서 RandomForestClassifier는 해당 문제에서는 조금 낮은 성능을 보여줄것이라 판단을 했고 이에 캐글 점수로 성능을 분석해본 결과 LogisticRegression의 성능이 더 좋았다. LogisticRegression의 성능을 더 높이기 위해서 파라미터를 설정해보았지만 성능이 더 높아지기 보다는 낮아지거나 같았다.